In [1]:
import pandas as pd

In [68]:
# kospi(유가증권)
#kospi_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=stockMkt', index_col=["종목코드"], header=0)[0]
kospi_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=stockMkt', header=0)[0]
print(kospi_df.shape)
# kosdaq
kosdaq_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=kosdaqMkt', header=0)[0]
print(kosdaq_df.shape)
# konexMkt
konex_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=konexMkt', header=0)[0]
print(konex_df.shape)

(777, 9)
(1269, 9)
(149, 9)


In [69]:
kospi_df['marketType'] = 'kospi'

In [70]:
kosdaq_df['marketType'] = 'kosdaq'

In [71]:
konex_df['marketType'] = 'konex'

In [76]:
frames = [kospi_df, kosdaq_df, konex_df]
stock_df = pd.concat(frames, ignore_index=True)
print(stock_df.shape)

(2195, 10)


In [77]:
stock_df['종목코드'] = stock_df['종목코드'].apply(lambda x: str(x).zfill(6))

In [78]:
feature_names = ['회사명', '종목코드', '업종', '결산월', 'marketType']
stock_train = stock_df[feature_names]

print(stock_train.shape)
stock_train.tail(5)

(2195, 5)


,회사명,종목코드,업종,결산월,marketType
2190,유디피,091270,소프트웨어 개발 및 공급업,12월,konex
2191,젠큐릭스,229000,의료용 기기 제조업,12월,konex
2192,청광종건,140290,토목 건설업,12월,konex
2193,탈모닷컴,284610,기타 화학제품 제조업,12월,konex
2194,피엔아이시스템,242350,"영화, 비디오물, 방송프로그램 제작 및 배급업",12월,konex


# 모든 종목 코드 갖고 오기

In [107]:
def get_all_codes():
    # kospi(유가증권)
    #kospi_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=stockMkt', index_col=["종목코드"], header=0)[0]
    kospi_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=stockMkt', header=0)[0]
    # kosdaq
    kosdaq_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=kosdaqMkt', header=0)[0]
    # konexMkt
    konex_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=konexMkt', header=0)[0]
    
    kospi_df['marketType'] = 'kospi'
    kosdaq_df['marketType'] = 'kosdaq'
    konex_df['marketType'] = 'konexMkt'
    print(kospi_df.shape, kosdaq_df.shape, konex_df.shape)
    
    frames = [kospi_df, kosdaq_df, konex_df]
    stock_df = pd.concat(frames, ignore_index=True)

    # 종목코드 6자리 맞추기
    stock_df['종목코드'] = stock_df['종목코드'].apply(lambda x: str(x).zfill(6))
    return stock_df

In [103]:
codes = get_all_codes()

(777, 10) (1269, 10) (149, 10)


In [104]:
print(codes.shape)
codes.tail()

(2195, 10)


,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,marketType
2190,유디피,091270,소프트웨어 개발 및 공급업,"IP카메라, 지능형영상분석SW",2014-12-12,12월,안정근,http://www.udptechnology.co.kr,서울특별시,konexMkt
2191,젠큐릭스,229000,의료용 기기 제조업,유방암예후진단키트,2015-10-27,12월,조상래,http://www.gencurix.com,서울특별시,konexMkt
2192,청광종건,140290,토목 건설업,"관도급, BTL, 자체사업",2013-12-20,12월,허숭,http://www.chungkwang.co.kr,서울특별시,konexMkt
2193,탈모닷컴,284610,기타 화학제품 제조업,탈모 샴푸 및 헤어케어 제품 등,2017-12-22,12월,장기영,http://www.talmo.com,서울특별시,konexMkt
2194,피엔아이시스템,242350,"영화, 비디오물, 방송프로그램 제작 및 배급업","3D 애니메이션, 3D VR 시뮬레이터 및 콘텐츠",2016-12-22,12월,신재중,http://www.pnisys.com,제주특별자치도,konexMkt


In [109]:
# 재무제표 년/월 얻어오기
import re
from datetime import datetime

def get_date_str(s):
    date_str = ''
    r = re.search("\d{4}/\d{2}", s)
    if r:
        date_str = r.group()

    return date_str

In [117]:
def get_financial_statement(code, ifrs_type=4, unit='Y'):
    url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=%s&fin_typ=%s&freq_typ=%s'
    url = url_tmpl % (code, ifrs_type, unit) # 삼성전자, 4(IFRS 연결), Y:년 단위
    print(url)

    dfs = pd.read_html(url, encoding='utf8')
    df = dfs[0]
    
    # multi columns 제거
    cols = list(df.columns.droplevel(0))
    cols = [get_date_str(col) for col in cols]
    
    df = df.set_index('주요재무정보')
    
    # 컬럼이름 변경
    df.columns = cols[0:-1]
    
    # index 변환
    indexes = []
    for i in range(0, df.index.size):
        indexes.append(df.index[i][0])
        
    new_index_dict = dict(zip(df.index, indexes))
    df.rename(new_index_dict, axis='index', inplace=True)
    
    # NaN을 0.0 숫자로 바꾸기
    df.fillna(0, inplace=True)
    
    return df

In [116]:

df.head()

http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=139130&fin_typ=4&freq_typ=Y


,2012/12,2013/12,2014/12,2015/12,2016/12,2017/12,2018/12,2019/12
주요재무정보,,,,,,,,
매출액,24602.0,24139.0,22494.0,35077.0,36550.0,0.0,0.0,0.0
영업이익,3730.0,3390.0,3239.0,3430.0,3869.0,4235.0,4784.0,5131.0
영업이익(발표기준),3730.0,3390.0,3239.0,3430.0,3869.0,0.0,0.0,0.0
세전계속사업이익,3630.0,3293.0,3172.0,3434.0,3878.0,4349.0,4927.0,5177.0
당기순이익,2741.0,2445.0,2438.0,3083.0,3019.0,3324.0,3706.0,3857.0


In [118]:
for idx, row in codes.iterrows():
    if idx % 100 == 0:
        df = get_financial_statement(codes.loc[idx]['종목코드'], 4, 'Y')
        df.head(3)

http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=139130&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=000540&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=096760&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=010060&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=005610&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=018670&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=006370&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=069460&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=153460&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=049630&fin_typ=4&freq_typ=Y
http://companyinfo.s